In [1]:
from selenium import webdriver

#browser = webdriver.Chrome()
#browser.get('http://seleniumhq.org/')

In [2]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time
import requests

try:
    url = None
    browser = webdriver.Chrome()

    browser.get('http://finance.yahoo.com')
    assert 'Yahoo' in browser.title

    elem = browser.find_element_by_id('yfin-usr-qry')  # Find the search box
    elem.send_keys('pacb') 
    time.sleep(5)
    stockPg = elem.send_keys(Keys.RETURN)
    time.sleep(5)

    pressRelease = browser.find_element(By.XPATH, '//span[text()="Press Releases"]').click()
    time.sleep(5)

    #newsLink = browser.find_element_by_link_text("Pacific Biosciences")
    newsLink = browser.find_element_by_class_name("StretchedBox")
    newsLink.click()
    print(browser.current_url)
    url = browser.current_url
    time.sleep(5)
except Exception as e:
    print(e)
    browser.quit()

browser.quit()

if url:
    r = requests.get(url)
    text = r.content
    print(text)

#browser.quit()

https://finance.yahoo.com/news/pacific-biosciences-california-inc-present-130000900.html
b'<!doctype html><html id=atomic class="Fz(62.5%) Pos(r)   desktop bktfinance-US-en-US-def ua-undefined ua-undefined" lang=en-US><head><script>\n        window.performance.mark(\'PageStart\');\n        document.documentElement.className += \' JsEnabled jsenabled\';\n        /**\n        * Empty darlaOnready method, to avoid JS error.\n        * This can happen when Async Darla JS file is loaded earlier than Darla Proxy JS.\n        * This method will be overridden by Darla Proxy\n        */\n        window.darlaOnready = function() {};\n        </script><title>Pacific Biosciences of California, Inc. to Present at Upcoming Investor Conferences</title><meta http-equiv=content-type content="text/html, charset=utf-8"><meta http-equiv=x-dns-prefetch-control content=on><meta http-equiv=X-UA-Compatible content=chrome=1><meta name="oath:guce:consent-host" content="guce.yahoo.com"><meta name="news_keywords"

In [4]:
from selenium import webdriver
url = "https://finance.yahoo.com/quote/NIO/press-releases?p=NIO"

links =[]
for i in range(1,2):
    #define a driver
    driver = webdriver.Chrome() #use the path in your computer
    driver.implicitly_wait(5)
    #get request
    driver.get(url)
    #keep clicking each news article and grab their url
    #elem = driver.find_element_by_class_name("StretchedBox")
    elem = driver.find_element_by_class_name("Cf")
    #get the URL after click
    elem.click()
    #store in a list
    links.append(driver.current_url)

print(links)

['https://finance.yahoo.com/quote/NIO/press-releases?p=NIO']


In [2]:
#title = 'Pacific Biosciences of California, Inc. Announces First Quarter 2021 Financial Results'
#title = 'Enterprise Reports First Quarter 2021 Earnings'
from datetime import datetime
from datetime import timedelta
import dateutil

def is_earningReport(title, earningDate, newsDate):
    print(earningDate, newsDate)
    temp = earningDate
    if newsDate != temp:
        temp = temp + timedelta(days=1)
        if newsDate != temp:
            temp = earningDate
            temp = temp + timedelta(days=-1)
            if newsDate != temp:
                return False
    
    words1 = ["Announces", "Reports"]
    words2 = ["Earnings", "Results"]
    if any(word in title for word in words1) and any(word in title for word in words2):
        #print(True)
        return True
    return False

In [3]:
newsDateStr = "2021-04-29 15:13:44.581375"
def get_newsMarketTime(newsDateStr):
    newsDate = datetime.strptime(newsDateStr, '%Y-%m-%d %H:%M:%S.%f')
    if (newsDate.hour > 12):
        return "AH"
    else:
        return "PM"

In [4]:
title = 'Pacific Biosciences of California, Inc. Announces First Quarter 2021 Financial Results'
newsDateArr = '2021-04-29 15:13:44.581375'.split(" ")
newsDateStr = newsDateArr[0]
newsDate = datetime.strptime(newsDateStr, '%Y-%m-%d')
earningDateStr = '2021-04-28'
earningDate = datetime.strptime(earningDateStr, '%Y-%m-%d')
is_earningReport(title, earningDate, newsDate)

2021-04-28 00:00:00 2021-04-29 00:00:00


True

In [5]:
def get_newsDate(l):
    #print(l)
    interval = l.split(" ")
    num = interval[0]
    date = datetime.now()
    newsDate = None
    #print(date)
    if num == "yesterday":
        newsDate = date + timedelta(days=-1)
        newsDate = newsDate + timedelta(hours=-9.30) #For US Eastern time
    elif interval[1] == "days":
        newsDate = date + timedelta(days=-int(interval[0]))
        newsDate = newsDate + timedelta(hours=-9.30) #For US Eastern time
        #print(newsDate)
    elif interval[1] == "hours":
        newsDate = date + timedelta(hours=-int(interval[0]))
        newsDate = newsDate + timedelta(hours=-9.30) #For US Eastern time
        #print(newsDate)
    elif interval[1] == "months":
        a_month = dateutil.relativedelta.relativedelta(months=int(interval[0]))
        newsDate = date - a_month
    else:
        newsDate = date
        newsDate = newsDate + timedelta(hours=-9.30) #For US Eastern time
        #print(newsDate)
    return newsDate

In [6]:
import time
from selenium import webdriver
from bs4 import BeautifulSoup


def get_news_list(stock, source="Yahoo"):
    if(source == "Benzinga"):
        if stock == "NEWS":
            url = "https://www.benzinga.com/news/"
        else:
            url = "https://www.benzinga.com/quote/" + stock
            
        news_list = list()
        driver = webdriver.Chrome()
        driver.get(url)
        time.sleep(5)
        
        soup = BeautifulSoup(driver.page_source, "html.parser")
        
        if stock == "NEWS":
            for t in soup.find_all(class_="listing-long-content"):
                #print (t)
                news = dict()
                a_tag = t.find("a", href=True)
                if(a_tag):
                    #print(a_tag.text)
                    text = a_tag.text
                    news["Title"] = text
                    news_list.append(news)
            driver.quit()
            return news_list

        for t in soup.find_all(class_="py-2 content-headline"):
            #print(t, t.text)
            a_tag = t.find("a", href=True)
            news = dict()
            if a_tag:
                #print(a_tag.text)
                text = a_tag.text
                news["Title"] = text
                news_list.append(news)
        driver.quit()
        return news_list
    
    # Default for Yahoo news
    if (stock == "NEWS"):
        url = "https://finance.yahoo.com/news/"
    else:
        url = "https://finance.yahoo.com/quote/" + stock + "?p=" + stock
        

    driver = webdriver.Chrome()
    driver.get(url)
    time.sleep(2)  # Allow 2 seconds for the web page to open
    scroll_pause_time = 2 
    screen_height = driver.execute_script("return window.screen.height;")   # get the screen height of the web

    i = 1
   
    SCROLL_PAUSE_TIME = 0.5
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(SCROLL_PAUSE_TIME)
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

    ##### Extract Article Titles #####
    news_list = list()
    soup = BeautifulSoup(driver.page_source, "html.parser")
    for t in soup.find_all(class_="Cf"):
        #print(t)
        news = dict()
        a_tag = t.find("a", class_="Fw(b)", href=True)
        #a_tag = t.find("u", class_="StretchedBox")
        if a_tag:
            #print(a_tag)
            text = a_tag.text
            news["Title"] = text
            news["Link"] = "https://finance.yahoo.com" + a_tag["href"]
            news["Provider"] = None
            news["Date"] = None
            news["Content"] = None
        
            provider = t.find("div", class_="C(#959595)")
            if provider:
                c = provider.text
                ctx = (c.encode("ascii", "replace").decode())
                l = ctx.split("?")
                news["Provider"] = l[0]
                news["Date"] = str(get_newsDate(l[1]))
            else:
                continue
        
            #p_content = t.find("p", class_="Fz(14px)")
            #if p_content:
            #    c = p_content.text
            #    news["Content"] = c
        
            newsDateArr = news["Date"].split(" ")
            newsDateStr = newsDateArr[0]
            newsDate = datetime.strptime(newsDateStr, '%Y-%m-%d')
            #if not is_earningReport(news["Title"], earningDate, newsDate):
            #    continue
            #else:
            news_list.append(news)
            #break
    #print(news_list)
    driver.quit()
    return news_list

In [7]:
import hashlib
import pandas as pd
import os
import time

path = "stocks_csv.csv"
d = pd.read_csv(path)
stocks = []
stocks.append("NEWS")
stocks = stocks + list(d["Ticker"])

#stocks = ["NIO", "BMY"]
source = ["Benzinga", "Yahoo"]
while True:
    for index, stock in enumerate(stocks):
        if type(stock) is not str:
            continue
        for src in source:
            #if (src == "Yahoo"):
            #    continue
            print("GET NEWS FOR STOCK {} from source {}".format(stock, src))
            news_list = get_news_list(stock, src)
            #print(news_list)
            for index, news in enumerate(news_list):
                hash_object = hashlib.sha1(news["Title"].encode("UTF-8"))
                hex_dig = hash_object.hexdigest()
                #dirc = "./News/unsup/"
                filename = "./News/allnews/" + stock + "_" + hex_dig + ".txt"
                ufilename = "./News/unsup/" + stock + "_" + hex_dig + ".txt"
                if news["Title"] is None:
                    continue
                print(hex_dig, news["Title"], filename)
                if os.path.isfile(filename):
                    print("File {} already exists".format(filename))
                    break
                with open(filename, 'w', encoding = 'utf-8') as f:
                    f.write(news["Title"])
                with open(ufilename, 'w', encoding = 'utf-8') as fu:
                    fu.write(news["Title"])
    break
    #print("DONEEEEEEEEEEEEEE, sleeping for 1 hour")
    #time.sleep(3600)

GET NEWS FOR STOCK NEWS from source Benzinga
b1e155f92da031c4d0c21bdb3a739be29cb18eae 5 Value Stocks In The Consumer Defensive Sector ./News/allnews/NEWS_b1e155f92da031c4d0c21bdb3a739be29cb18eae.txt
06cbdd1ce712522e6f3bab90f011b1a775709478 Grocers Love Online Sales. They Hate The Impact It Is Having On Margins ./News/allnews/NEWS_06cbdd1ce712522e6f3bab90f011b1a775709478.txt
a30d51dbd1563af8d45752c88a3fda101c14437b Wells Fargo Initiates Chart Industries With Overweight Rating, BTIG Raises PT ./News/allnews/NEWS_a30d51dbd1563af8d45752c88a3fda101c14437b.txt
cc27239c2edaad7dbb552b47d40d676a9655e040 5 Value Stocks In The Basic Materials Sector ./News/allnews/NEWS_cc27239c2edaad7dbb552b47d40d676a9655e040.txt
c0a915d480d571dd08a16fb3a25e83abe5bfb1c1 Inflation Appears To Be Setting The Tone For Stocks Going Forward ./News/allnews/NEWS_c0a915d480d571dd08a16fb3a25e83abe5bfb1c1.txt
68bbef9ec9a108d85363eb982623358f3c1a478c 3 Stocks Insiders Are Buying ./News/allnews/NEWS_68bbef9ec9a108d85363eb9826

53aa2b0192e1042d0c098bd8fba8308b760de23c Got $5,000? Here Are 5 Energy Stocks to Buy and Hold for the Long Term ./News/allnews/EPD_53aa2b0192e1042d0c098bd8fba8308b760de23c.txt
dbbc63e1af99b190debcfe4df35be6c6f5c26e3f Enterprise Products Partners (EPD) Gains As Market Dips: What You Should Know ./News/allnews/EPD_dbbc63e1af99b190debcfe4df35be6c6f5c26e3f.txt
File ./News/allnews/EPD_dbbc63e1af99b190debcfe4df35be6c6f5c26e3f.txt already exists
GET NEWS FOR STOCK ENB from source Benzinga
f911b35eadaf7a16d02d3eaa1b6a9518325090d8 Shares of several companies in the broader energy sector are trading lower amid overall market weakness as stocks across sectors fall on COVID-19 concerns and China volatility. Investors also await the Feds two-day meeting beginning on Tuesday. ./News/allnews/ENB_f911b35eadaf7a16d02d3eaa1b6a9518325090d8.txt
File ./News/allnews/ENB_f911b35eadaf7a16d02d3eaa1b6a9518325090d8.txt already exists
GET NEWS FOR STOCK ENB from source Yahoo
df28c9a106c0eb77db1b6a25ac5910af5eb333

a235aa3b2ee82326b3f9ae56728865ae3124a861 Nike analyst says supply chain issues could be solved ‘well into 2022’ ./News/allnews/NKE_a235aa3b2ee82326b3f9ae56728865ae3124a861.txt
99df99a7ae1269d53d26cd8bd924c997cda53512 Stock Market Narrowly Mixed As China, Bitcoin Woes Weigh; Nike Tumbles ./News/allnews/NKE_99df99a7ae1269d53d26cd8bd924c997cda53512.txt
d072a4f32a9f55a2129fb4a5c3ef1b6397a13668 US STOCKS-Wall Street rally fizzles out as Evergrande worries persist; Nike drops ./News/allnews/NKE_d072a4f32a9f55a2129fb4a5c3ef1b6397a13668.txt
3eaea662e3566991ca72a32664b96f629682faa9 Dow Jones Reverses As Evergrande Misses Interest Payment; Bitcoin Tumbles On China Ban ./News/allnews/NKE_3eaea662e3566991ca72a32664b96f629682faa9.txt
da69cd976837d7818280d6c11dd43b2c4592ceb4 NIKE (NKE) Q1 Earnings Beat, Sales Miss on Supply Constraints ./News/allnews/NKE_da69cd976837d7818280d6c11dd43b2c4592ceb4.txt
56865f8e80947c262aade25ab722aba5509d746a Nike Stock Tumbles On Sales Miss, Weak Guidance Amid Footwear

5feb496cfdfee7c3e2fcb4ddaeaf9e6f78ba2ae3 Fulgent Genetics to Partner with Houston Health Department and Houston-Area Public Schools on Back-to-School COVID-19 Testing Program ./News/allnews/FLGT_5feb496cfdfee7c3e2fcb4ddaeaf9e6f78ba2ae3.txt
973cd7c5d655d400f84d24e4d034b1f16ef62a5e 3 Unstoppable Healthcare Stocks to Buy With Just $1,000 ./News/allnews/FLGT_973cd7c5d655d400f84d24e4d034b1f16ef62a5e.txt
File ./News/allnews/FLGT_973cd7c5d655d400f84d24e4d034b1f16ef62a5e.txt already exists
GET NEWS FOR STOCK HZNP from source Benzinga
2ccd9562fc46b066dea59dd60dad695972fadff3 Horizon Therapeutics Highlights Data Showing UPLIZNA Also Produces B-Cell Depletion In African Americans With Neuromeylitis Optica Spectrum Disorder ./News/allnews/HZNP_2ccd9562fc46b066dea59dd60dad695972fadff3.txt
File ./News/allnews/HZNP_2ccd9562fc46b066dea59dd60dad695972fadff3.txt already exists
GET NEWS FOR STOCK HZNP from source Yahoo
8a4367192c0a42cf8149e37a35294b57c02027df Horizon Therapeutics plc Announces Agenda for

3a8154a21f7500d7acf5f8cdbdbf55fa7dfbdecd Tesla's German landing will fuel competition - Volkswagen CEO ./News/allnews/TSLA_3a8154a21f7500d7acf5f8cdbdbf55fa7dfbdecd.txt
64bb5428b93a36dd11fa562cc47acc7ae09d5b48 Weekend reads: Retirement locations in 6 states ./News/allnews/TSLA_64bb5428b93a36dd11fa562cc47acc7ae09d5b48.txt
576994c9a388f0c34a511bfd25d1c255d9ba6f37 Can Intuitive Surgical Be the Tesla of Healthcare? ./News/allnews/TSLA_576994c9a388f0c34a511bfd25d1c255d9ba6f37.txt
e26c862cf7c5dc1f03a2cac925eb4898631db472 Move over, Cathie Wood: 3 picks from Goldman Sachs' new ETF that could smash Ark ./News/allnews/TSLA_e26c862cf7c5dc1f03a2cac925eb4898631db472.txt
3eaea662e3566991ca72a32664b96f629682faa9 Dow Jones Reverses As Evergrande Misses Interest Payment; Bitcoin Tumbles On China Ban ./News/allnews/TSLA_3eaea662e3566991ca72a32664b96f629682faa9.txt
ef4afd17726aad6ded735f45388d432ea9f5e150 Is Lucid Motors Stock A Buy Right As Lucid Air Gets EPA Rating Of 520 Miles Per Charge? ./News/allne

...

6ea37d93902e7cb2b1d3670d2a559c4685821c90 Salesforce COO: 'If I started a company today, I'd start with my digital headquarters' ./News/allnews/CRM_6ea37d93902e7cb2b1d3670d2a559c4685821c90.txt
103a02c1d58e7f3fe0b12cb12845ac2c8fda9493 Is Salesforce Stock A Buy? Software Maker Raises Fiscal 2022 Revenue Outlook ./News/allnews/CRM_103a02c1d58e7f3fe0b12cb12845ac2c8fda9493.txt
0910fafcfb1f24533ab749005d81fa1214b02ff6 Stock Market Rally Roars Back On Fed Decision, Evergrande News; Salesforce, FedEx, Facebook, Nike, Adobe In Focus: Weekly Review ./News/allnews/CRM_0910fafcfb1f24533ab749005d81fa1214b02ff6.txt
c739946cc1853570224ec51c490725c479532177 Dow Jones Futures Fall, Bitcoin Tumbles On China Cryptocurrency Ban; Salesforce Leads 17 Stocks Flashing Buys ./News/allnews/CRM_c739946cc1853570224ec51c490725c479532177.txt
eaf6dffa5f5477fdda07e71a95bf58bece8a7df7 Why Analysts See Up To 32% Upside In Salesforce? ./News/allnews/CRM_eaf6dffa5f5477fdda07e71a95bf58bece8a7df7.txt
File ./News/allnews/CRM

32e1e32c19f16d894acb8168d7b7caad620c7d48 FedEx Plays Game Of Chicken With Parcel Shippers' Budgets ./News/allnews/AMZN_32e1e32c19f16d894acb8168d7b7caad620c7d48.txt
f97ddaccf47a69548c3106e8e20b77e6feeec648 Friday's Market Minute: Takeaways From Another Volatile Earnings Season ./News/allnews/AMZN_f97ddaccf47a69548c3106e8e20b77e6feeec648.txt
98ea70d2e2e8a967df9014adbaab347b42a330ab Cloudflare Gets An Analyst Downgrade - Read Why ./News/allnews/AMZN_98ea70d2e2e8a967df9014adbaab347b42a330ab.txt
e2cfdd10433b8fb39d8572baa175c26ea7be5b07 'Amazon Working on Large Wall Echo, Sound Bar and New Auto Device' -Bloomberg ./News/allnews/AMZN_e2cfdd10433b8fb39d8572baa175c26ea7be5b07.txt
587489fed39a54e6f7a583966c2216385ebb19e2 Amazon Heats Up Streaming Rivalry With Disney, Netflix In India ./News/allnews/AMZN_587489fed39a54e6f7a583966c2216385ebb19e2.txt
eeb0328318c144342580ceed38e7dc0124ace897 Disney's Space 220 Restaurant Features NASA-Inspired Glimpses With Stellarvator, Orbit View And More ./News/a

b5e28903f8cfd5e44af4e9162026edd3b80e7bc4 Zynga Option Alert: Nov 19 $8 Calls Sweep (10) near the Ask: 890 @ $0.411 vs 2490 OI; Earnings 11/3 After Close [est] Ref=$7.5601 ./News/allnews/ZNGA_b5e28903f8cfd5e44af4e9162026edd3b80e7bc4.txt
File ./News/allnews/ZNGA_b5e28903f8cfd5e44af4e9162026edd3b80e7bc4.txt already exists
GET NEWS FOR STOCK ZNGA from source Yahoo
9cd12ceaf3d08d1d9424a431a6090851ae6fb2f2 Got $1,000? Here Are 3 Stocks to Buy for the Long Term ./News/allnews/ZNGA_9cd12ceaf3d08d1d9424a431a6090851ae6fb2f2.txt
e893fea7111b6a2248de2374ac670d02e2d80c83 3 Top Stocks Under $10 a Share ./News/allnews/ZNGA_e893fea7111b6a2248de2374ac670d02e2d80c83.txt
e669bd36b1a57c623e6e25bb639d811bc92d6703 Zynga (ZNGA) Stock Sinks As Market Gains: What You Should Know ./News/allnews/ZNGA_e669bd36b1a57c623e6e25bb639d811bc92d6703.txt
File ./News/allnews/ZNGA_e669bd36b1a57c623e6e25bb639d811bc92d6703.txt already exists
GET NEWS FOR STOCK LULU from source Benzinga
5113c84df64a26b9ee66df1b24e307632bef425a

b7e83578070b30a9b17ca39c4fd4db620594ff13 NYC Tech Push Gains Steam With Microsoft, Roku in Hunt for Space ./News/allnews/JPM_b7e83578070b30a9b17ca39c4fd4db620594ff13.txt
fb00a56a6ad7128bbf5fa1cf1876300bb51ca712 Bank Stock Roundup: Expansion Plans of USB, JPM, C & Asset Cap on WFC in Focus ./News/allnews/JPM_fb00a56a6ad7128bbf5fa1cf1876300bb51ca712.txt
d8e360f5a145432a0668806abbc76f1cab1599d0 Tycoon Richard Li’s Insurer FWD Files for Long-Awaited IPO ./News/allnews/JPM_d8e360f5a145432a0668806abbc76f1cab1599d0.txt
289b9dbd6815b38f9c8a8ab8c25956da192b8218 Treasury Yields Leap as Traders Accelerate Fed Rate Liftoff Bets ./News/allnews/JPM_289b9dbd6815b38f9c8a8ab8c25956da192b8218.txt
d2c673ae384908a20f0bcda0e0666386db5a0abf JPMorgan agrees to pay $15.7 million to settle spoofing lawsuit ./News/allnews/JPM_d2c673ae384908a20f0bcda0e0666386db5a0abf.txt
d1c27dd2723015cc9159afe28980294512048f96 JPMorgan agrees to pay $15.7 mln to settle spoofing lawsuit ./News/allnews/JPM_d1c27dd2723015cc9159afe

4b394328d08eaac45b58fbd0505aafd7d0e31dc2 BP temporarily closes some petrol sites amid lorry driver shortage ./News/allnews/BP_4b394328d08eaac45b58fbd0505aafd7d0e31dc2.txt
8176b1cf977e8f777594c52bea3a55488b9b914b BP and Tesco close pumps and ration fuel as deliveries hit by driver shortage ./News/allnews/BP_8176b1cf977e8f777594c52bea3a55488b9b914b.txt
536544fda1e7aab505c4b180ffa271f481ee913d BP may ration gasoline in U.K. due to truck driver shortage: report ./News/allnews/BP_536544fda1e7aab505c4b180ffa271f481ee913d.txt
2f7164589eb37ed825a463e1aefed7ffbf55a8bf BP (BP) Outpaces Stock Market Gains: What You Should Know ./News/allnews/BP_2f7164589eb37ed825a463e1aefed7ffbf55a8bf.txt
File ./News/allnews/BP_2f7164589eb37ed825a463e1aefed7ffbf55a8bf.txt already exists
GET NEWS FOR STOCK DGRO from source Benzinga
GET NEWS FOR STOCK DGRO from source Yahoo
982762303a0e1f7a4d8cd74d3393000019ba455b Big Shifts In ETF Ownership ./News/allnews/DGRO_982762303a0e1f7a4d8cd74d3393000019ba455b.txt
File ./Ne

In [ ]:
import spacy
import time
from datetime import datetime
import os
import hashlib

chkp = "./News/checkpoint.txt"
default_time = "Sun May 16 23:03:21 2020"
allnews = "./News/allnews/"
unsupnews = "./News/unsup/"

nlp = spacy.load("en_core_web_sm")
nlp.add_pipe(nlp.create_pipe('sentencizer'))

if not os.path.isfile(chkp):
    with open(chkp, 'wt', encoding="utf-8") as f:
        f.write(default_time)

with open(chkp, 'r', encoding="utf-8") as f:
    last_read = f.readline()

print(last_read)

last_read_date = datetime.strptime(last_read, "%a %b %d %H:%M:%S %Y")
        
summary_unsup_path = "./unsup"
MIN_WORDS_EACH_LINE = 5
total_sentence = 0

for filename in os.listdir(summary_unsup_path):
    fn = os.path.join(summary_unsup_path, filename)
    last_mod = time.ctime(os.path.getmtime(fn))
    last_mod_date = datetime.strptime(last_mod, "%a %b %d %H:%M:%S %Y")
    if(last_mod_date > last_read_date):
        print("File to be processed {}".format(fn))
        with open(fn, 'r', encoding="utf-8") as f:
            text = f.read()
        doc = nlp(text)
        sentences = [sent.string.strip() for sent in doc.sents]
        
        #print("File {} has {} sentences".format(fn, len(sentences)))
        cnt = 0
        sent_cnt = len(sentences)
        
        seps = filename.split("-")
        stock = seps[0]
        
        for line in sentences:
            words = line.split(" ")
            #print(len(words), line)
            if len(words) <= MIN_WORDS_EACH_LINE:
                cnt = cnt + 1
                continue
            hash_object = hashlib.sha1(line.encode("UTF-8"))
            hex_dig = hash_object.hexdigest()
            allnfn = allnews + stock + "_" + hex_dig + "_REPT" + ".txt"
            unsupfn = unsupnews + stock + "_" + hex_dig + "_REPT" + ".txt"
            
            if os.path.isfile(allnfn):
                print("File {} already exists".format(allnfn))
                break
            
            with open(allnfn, 'w', encoding = 'utf-8') as fa:
                fa.write(line)
            with open(unsupfn, 'w', encoding = 'utf-8') as fu:
                fu.write(line)
            
            print("FILEs {} {} created".format(allnfn, unsupfn))
            
        print("File {} has {} valid sentences for stock {}".format(fn, (sent_cnt - cnt), stock))
        total_sentence = total_sentence + (sent_cnt - cnt)
    
print("Total files created {}".format(total_sentence))
        
with open(chkp, 'w', encoding="utf-8") as f:
    f.write(time.ctime())

In [11]:
import os
import shutil
# assign directory
directory = "./News/unsup/"
pos = "./News/train/pos/"
neg = "./News/train/neg/"
neu = "./News/train/neu/"

neuStr = ("*neu")
posStr = ("*pos")
negStr = ("*neg")

neuRpl = (" - *neu")
posRpl = (" - *pos")
negRpl = (" - *neg")

# itrate over files in
# that directory
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    # checking if it is a file
    #if os.path.isfile(f):
    #    print(f)
    with open(f, 'r', encoding="utf-8") as fl:
        line = fl.readline()
 
    if (line):
        if (neuStr in line):
            #print(line)
            df = os.path.join(neu, filename)
        elif (posStr in line):
            #print(line)
            df = os.path.join(pos, filename)
        elif (negStr in line):
            #print(line)
            df = os.path.join(neg, filename)
        else:
            continue
            
        if os.path.isfile(df):
            print("File {} Exists".format(df))
            continue
        shutil.move(f, df)
        print(df)
        
        with open(df, 'rt', encoding="utf-8") as dfl:
            rl = dfl.read()
            if (neuStr in rl):
                rl = rl.replace(neuRpl, "")
            if (posStr in rl):
                rl = rl.replace(posRpl, "")
            if (negStr in rl):
                rl = rl.replace(negRpl, "")
                
        with open(df, 'wt', encoding="utf-8") as dfl:
            dfl.write(rl)



./News/train/pos/CPNG_f479945b440c27a2f56b97292046d2953262135a.txt
File ./News/train/neu/PACB_44faa3acf722d5663f2b1e418d1043374f6e95c5.txt Exists
File ./News/train/neu/PACB_5ca00b550924e2e882b1009ecddc144cfe7f2ad5.txt Exists
./News/train/neu/PACB_663f0c3d011739eb04a5b493668723d9675fb78f.txt
File ./News/train/neg/PACB_92dfd49098bd4ecfff963be11095319277003ec9.txt Exists
File ./News/train/neu/PACB_b04d226d4b380d62c1b121d27ffda8127e3dbfb4.txt Exists
File ./News/train/pos/PACB_d3827923b4798a417153297e66abd8ec2d4a3688.txt Exists
File ./News/train/neg/PACB_f2ae485a7d75dea5e7e416186d7c4c2a50a0a559.txt Exists
./News/train/neu/XPEV_4478de45ea1af294476415290b0377435830139a.txt


In [254]:
s = "Nanox to Announce First Quarter 2021 Financial and Operating Results on Tuesday, May 11, 2021"
s1 = "Opendoor to Report First Quarter 2021 Financial Results on May 11, 2021 "
s3 = "Palantir Announces Date of First Quarter 2021 Earnings Release and Conference Call"
s4 = "Revolve Group, Inc. to Announce First Quarter 2021 Financial Results on May 6, 2021"
s5 = "Snowflake to Announce Financial Results for the First Quarter of Fiscal 2022 on May 26, 2021"
s6 = "Vir Biotechnology to Provide Corporate Update and Report First Quarter 2021 Financial Results on May 6, 2021"
s7 = "Wix to Announce First Quarter 2021 Results on May 12, 2021"
s8 = "XPeng to Report First Quarter 2021 Financial Results on Thursday, May 13, 2021"
s9 = "Zoom Video Communications to Release Financial Results for the First Quarter of Fiscal Year 2022"

In [13]:
import random
s = "./News/train/"
d = "./News/test/"
dirs = os.listdir(s)
print(dirs)

for sd in dirs:
    path = os.path.join(s, sd)
    dpath = os.path.join(d, sd)
    print(path, dpath)
    f2mvcnt = (int(len(os.listdir(path)) * 20 / 100))
    existing = (int(len(os.listdir(dpath))))
    print(f2mvcnt, existing)
    if(existing >= f2mvcnt):
        continue
    f2mvcnt = f2mvcnt - existing
    print("Files to be moved {}".format(f2mvcnt))
    cnt = 0
    movedfile = []
    while cnt < f2mvcnt:
        file = random.choice(os.listdir(path))
        if file not in movedfile:
            movedfile.append(file)
            cnt = cnt + 1
            sf = os.path.join(path, file)
            df = os.path.join(dpath, file)
            
            if not os.path.isfile(df):
                print(sf, df)
                shutil.move(sf, df)


['neg', 'neu', 'pos']
./News/train/neg ./News/test/neg
35 41
./News/train/neu ./News/test/neu
68 80
./News/train/pos ./News/test/pos
87 105
